In [1]:
import pandas as pd


In [2]:
df_hr_data = pd.read_xml('/Users/eshishmarina/PycharmProjects/HR_data_analysis/data/hr_data.xml')
df_A_office = pd.read_xml('/Users/eshishmarina/PycharmProjects/HR_data_analysis/data/A_office_data.xml')
df_B_office = pd.read_xml('/Users/eshishmarina/PycharmProjects/HR_data_analysis/data/B_office_data.xml')

Меняем ID сотрудников и устанавливаем новый индекс, объединяем три таблицы в одну

In [3]:
df_A_office['employee_office_id'] = 'A' + df_A_office['employee_office_id'].astype('str')
df_B_office['employee_office_id'] = 'B' + df_B_office['employee_office_id'].astype('str')

df_A_office.set_index('employee_office_id', inplace=True)
df_B_office.set_index('employee_office_id', inplace=True)


AB_offices = pd.concat([df_A_office, df_B_office])
all_merged = AB_offices.merge(df_hr_data, how='left', left_on='employee_office_id', right_on='employee_id', indicator=True)
all_merged = all_merged[all_merged.employee_id.notnull()]
all_merged.drop(columns=['_merge'], inplace=True)
all_merged.set_index('employee_id', inplace=True)


Департаменты Топ-10 сотрудников по кол-ву рабочего времени.
Группируем по сотруднику и департаменту, отбираем первые 10 по кол-ву раб часов. У результата получается двойной индекс(кортеж): ID работника + департамент. Достаем департамент из каждого кортежа

In [4]:
by_hours = all_merged.groupby(['employee_id', 'Department'])['average_monthly_hours'].sum().sort_values(ascending=False).head(10)
by_hours_indexes = by_hours.index.values
first_result = [tup[1] for tup in by_hours_indexes]
print(first_result)

['support', 'marketing', 'technical', 'hr', 'support', 'sales', 'hr', 'support', 'technical', 'technical']


Общее кол-во проектов в ИТ-департаменте у сотрудников с низкими зарплатами.
Делаем отбор по департаменту и уровню ЗП, суммируем кол-во проектов

In [5]:
second_result = all_merged.query("Department == 'IT' & salary == 'low'")['number_project'].sum()
print(second_result)

847


Последние оценки и уровень удовлетворенности сотрудников с ID A4, B7064, A3033.
Используем метод loc, по строкам фильтруем ID работника, по столбцам оценку работника

In [6]:
list_A4 = list(all_merged.loc['A4', ['last_evaluation', 'satisfaction_level']])
list_B7064 = list(all_merged.loc['B7064', ['last_evaluation', 'satisfaction_level']])
list_A3033 = list(all_merged.loc['A3033', ['last_evaluation', 'satisfaction_level']])
third_result = [list_A4, list_B7064, list_A3033]
print(third_result)

[[0.87, 0.72], [0.56, 0.36], [0.94, 0.93]]


Медианное кол-во проектов с группировкой по признаку 'left' (уволившиеся и не уволившиеся сотрудники)

In [7]:
median_num_of_projects = all_merged.groupby('left')['number_project'].agg('median').round(2).to_dict()

Кол-во сотрудников, которые работали более чем над 5 проектами с группировкой по признаку 'left'

In [8]:
summa = pd.pivot_table(all_merged, index='employee_id', columns='left', values='number_project', aggfunc=sum)
summa = summa[(summa[0] > 5) | (summa[1] > 5)]
not_left = summa[0].count().round(2)
left = summa[1].count().round(2)

Среднее и медианное количество отработанного времени с группировкой по признаку 'left'

In [9]:
median_time_spent = all_merged.groupby('left')['time_spend_company'].agg('median').round(2).to_dict()
mean_time_spent = all_merged.groupby('left')['time_spend_company'].agg('mean').round(2).to_dict()

Среднее и средне-квадратичное отклонение оценки с группировкой по признаку 'left'

In [10]:
mean_eval = all_merged.groupby('left')['last_evaluation'].agg('mean').round(2).to_dict()
std_eval = all_merged.groupby('left')['last_evaluation'].agg('std').round(2).to_dict()

Доля работников, у которых были рабочие происшествия с группировкой по признаку 'left'

In [11]:
all_merged['count_accident'] = all_merged.groupby('left')['Work_accident'].transform('sum')
all_merged['count_by_left'] = all_merged.groupby('left')['left'].transform('count')
all_merged['accident_pct'] = (all_merged['count_accident'] / all_merged['count_by_left']).round(2)
accident_pct_by_left = all_merged.groupby('left')['accident_pct'].mean().to_dict()

In [12]:
result = {('number_project', 'median'): median_num_of_projects,
 ('number_project', 'count_bigger_5'): {0: not_left, 1: left},
 ('time_spend_company', 'mean'): mean_time_spent,
 ('time_spend_company', 'median'): median_time_spent,
 ('Work_accident', 'mean'): accident_pct_by_left,
 ('last_evaluation', 'mean'): mean_eval,
 ('last_evaluation', 'std'): std_eval}
print(result)

{('number_project', 'median'): {0.0: 4.0, 1.0: 4.0}, ('number_project', 'count_bigger_5'): {0: 207, 1: 339}, ('time_spend_company', 'mean'): {0.0: 3.4, 1.0: 3.91}, ('time_spend_company', 'median'): {0.0: 3.0, 1.0: 4.0}, ('Work_accident', 'mean'): {0.0: 0.18, 1.0: 0.04}, ('last_evaluation', 'mean'): {0.0: 0.72, 1.0: 0.72}, ('last_evaluation', 'std'): {0.0: 0.16, 1.0: 0.2}}


Данные для сводной таблицы, где список департаментов соответствует условиям:
1. для неуволившихся сотрудников медианное кол-во отработанных часов у работников с высокой зарплатой меньше,
чем медианное кол-во отработанных часов у работников со средней зарплатой.

In [13]:
high_salary_hours_0 = all_merged.query("salary=='high' & left==0").groupby('Department')['average_monthly_hours'].median().rename('average_monthly_hours_high').reset_index()
medium_salary_hours_0 = all_merged.query("salary=='medium' & left==0").groupby('Department')['average_monthly_hours'].median().reset_index()

a = high_salary_hours_0.merge(medium_salary_hours_0)
a = a[a.average_monthly_hours_high < a.average_monthly_hours]
dep_list_0 = list(a['Department'])

2. для уволившихся сотрудников медианное кол-во отработанных часов у работников с низкой зарплатой меньше,
чем медианное кол-во отработанных часов у работников со высокой зарплатой.

In [14]:
low_salary_hours_1 = all_merged.query("salary=='low' & left==1").groupby('Department')['average_monthly_hours'].median().rename('average_monthly_hours_low').reset_index()
high_salary_hours_1 = all_merged.query("salary=='high' & left==1").groupby('Department')['average_monthly_hours'].median().reset_index()

b = low_salary_hours_1.merge(high_salary_hours_1)
b = b[b.average_monthly_hours_low < b.average_monthly_hours]
dep_list_1 = list(b['Department'])

dep_list = dep_list_0 + dep_list_1

all_merged_for_pivot1 = all_merged[all_merged.Department.isin(dep_list)]

In [19]:
first_pivot = pd.pivot_table(all_merged_for_pivot1, index='Department', columns=['left', 'salary'], values='average_monthly_hours', aggfunc='median')
print(first_pivot)

left           0.0                  1.0              
salary        high    low medium   high    low medium
Department                                           
IT           193.0  198.5  199.0  155.0  235.0  198.0
management   196.0  208.0  201.0  259.0  230.5  235.0
marketing    173.0  199.5  185.0  148.5  155.0  157.0
product_mng  172.0  198.5  202.0  149.0  218.0  154.5
sales        190.0  198.0  198.0  241.5  224.5  225.0
support      214.0  194.5  196.0  237.0  219.0  221.0
technical    193.0  197.0  202.0  157.5  244.0  232.0


Данные для сводной таблицы, где оценка сотрудника, которого повысили меньше, чем у того, которого не повысили.

In [16]:
promoted = all_merged.query("promotion_last_5years==1").groupby('time_spend_company')['last_evaluation'].mean().rename('promoted').reset_index()
not_promoted = all_merged.query("promotion_last_5years==0").groupby('time_spend_company')['last_evaluation'].mean().rename('not_promoted').reset_index()

c = promoted.merge(not_promoted)
c = c[c.not_promoted > c.promoted]
time_spend_company_included = list(c['time_spend_company'])
all_merged_for_pivot2 = all_merged[all_merged.time_spend_company.isin(time_spend_company_included)]

In [18]:
second_pivot = pd.pivot_table(all_merged_for_pivot2, index='time_spend_company', columns=['promotion_last_5years'], values=['last_evaluation', 'satisfaction_level'], aggfunc=['max', 'mean', 'min'])
print(second_pivot)

                                  max                                 \
                      last_evaluation       satisfaction_level         
promotion_last_5years               0     1                  0     1   
time_spend_company                                                     
2                                 1.0  0.99               1.00  0.94   
4                                 1.0  0.90               1.00  0.94   
5                                 1.0  0.91               1.00  0.82   
6                                 1.0  0.71               1.00  0.81   
10                                1.0  0.94               0.99  0.85   

                                 mean                                        \
                      last_evaluation          satisfaction_level             
promotion_last_5years               0        1                  0         1   
time_spend_company                                                            
2                            0.7162